#Objective

*   With a massive and disorganized amount of data extracted, it is necessary to carry out some transformations to make them consumable in Machine Learning models;
*   The expected output is an organized dataframe, with its data treated and cleaned.




# 1. Library imports

In [ ]:
import pandas as pd
import numpy as np
import re
import time

import bs4 as bs4
import json

import glob
import tqdm

#https://strftime.org/
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# 2. Data processing and feature creation

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Manually added the "y" column with the target for 609 random games.

In [ ]:
df = pd.read_csv("", index_col=0)
df = df[df['y'].notnull()]
df.shape

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Projetos/Recomendação jogos/games_raw_data_with_labels.csv", index_col=0)
df = df[df['y'].notnull()]
df.shape

(609, 10)

We can see here that to use the data we will need to clean it:

In [ ]:
df.head(20)

,product_title,y,metascore_wrap_highlight_metascore,userscore_wrap_feature_userscore,summary_detail_publisher,summary_detail_developer,summary_detail_release_data,summary_detail_product_genre,summary_detail_product_players,summary_detail_product_rating
0,The Legend of Zelda: Ocarina of Time\nNintendo 64,1.0,Metascore\n99\nUniversal acclaim\n- based on\n...,User Score\n9.1\n\nUniversal acclaim- based on...,Publisher:\nNintendo,Developer:\n\nNintendo,"Release Date:\nNov 23, 1998","Genre(s): Action Adventure, Fantasy",# of players:\n1 Player,Rating:\nE
1,Tony Hawk's Pro Skater 2\nPlayStation,1.0,Metascore\n98\nUniversal acclaim\n- based on\n...,User Score\n7.4\n\nMixed or average reviews- b...,Publisher:\nActivision,Developer:\n\nNeversoft Entertainment,"Release Date:\nSep 20, 2000","Genre(s): Sports, Alternative, Skateboarding",# of players:\n1-2,Rating:\nT
2,Babylon's Fall,0.0,Metascore\n97\nUniversal acclaim\n- based on\n...,User Score\n8.2\n\nGenerally favorable reviews...,Publisher:\nZA/UM,Developer:\n\nZA/UM,"Release Date:\nMar 30, 2021","Genre(s): Role-Playing, General, Western-Style",# of players:\nNo Online Multiplayer,Rating:\nM
3,Jet Grind Radio\nDreamcast,1.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n8.3\n\nGenerally favorable reviews...,Publisher:\nSega,Developer:\n\nSmilebit,"Release Date:\nOct 30, 2000","Genre(s): Action, Platformer, 3D",NaN,Rating:\nT
4,Metal Gear Solid\nPlayStation,1.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n9.2\n\nUniversal acclaim- based on...,Publisher:\nKonami,Developer:\n\nKCEJ,"Release Date:\nOct 21, 1998","Genre(s): Action Adventure, Modern",# of players:\n1 Player,Rating:\nM
5,Babylon's Fall,0.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n9.1\n\nUniversal acclaim- based on...,Publisher:\nLucasArts,Developer:\n\nLucasArts,"Release Date:\nOct 14, 1998","Genre(s): Adventure, General, General, 3D, Thi...",# of players:\nNo Online Multiplayer,Rating:\nT
6,Tom Clancy's Splinter Cell Chaos Theory\nXbox,1.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n9.1\n\nUniversal acclaim- based on...,Publisher:\nUbisoft,Developer:\n\nUbisoft Montreal,"Release Date:\nMar 28, 2005","Genre(s): Action Adventure, Modern",# of players:\n4 Online,Rating:\nM
7,Resident Evil Code: Veronica\nDreamcast,1.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n9.0\n\nUniversal acclaim- based on...,Publisher:\nCapcom,Developer:\n\nNexTech,"Release Date:\nFeb 29, 2000","Genre(s): Action Adventure, Survival",# of players:\nNo Online Multiplayer,Rating:\nM
8,Burnout 3: Takedown\nXbox,1.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n7.7\n\nGenerally favorable reviews...,Publisher:\nEA Games,Developer:\n\nCriterion Games,"Release Date:\nSep 7, 2004","Genre(s): Driving, Racing, Arcade",# of players:\n8 Online,Rating:\nT
9,Babylon's Fall,1.0,Metascore\n94\nUniversal acclaim\n- based on\n...,User Score\n8.6\n\nGenerally favorable reviews...,Publisher:\nBlizzard Entertainment,Developer:\n\nBlizzard Entertainment,"Release Date:\nDec 31, 1996","Genre(s): Role-Playing, Action RPG, Action RPG",# of players:\nUp to 4,Rating:\nM


In [ ]:
df.isna().sum()

product_title                           0
y                                       0
metascore_wrap_highlight_metascore      0
userscore_wrap_feature_userscore        0
summary_detail_publisher                1
summary_detail_developer                2
summary_detail_release_data             0
summary_detail_product_genre            0
summary_detail_product_players        117
summary_detail_product_rating          35
dtype: int64

In [ ]:
df_limpo = pd.DataFrame(index=df.index)

## Using regex we will extract only the desired information in each column, saving it in a new dataframe.

In [ ]:


#date

release = df['summary_detail_release_data'].str.extract(r"((?<=Release Date:\n).*)", expand=False)
df_limpo['release_data'] = pd.to_datetime(release, format="%b %d, %Y")

#tittle

titulo = df['product_title'].str.extract("([^\n]*)", expand=False)
df_limpo['tittle'] = titulo

#userscore
score_user = df['userscore_wrap_feature_userscore'].str.extract("(\w\.\w)", expand=False)
df_limpo['userscore'] = score_user


#metascore
score_meta = df['metascore_wrap_highlight_metascore'].str.extract("((?<=Metascore\n).*)", expand=False)
df_limpo['metascore'] = score_meta

#genre
genre = df['summary_detail_product_genre'].str.extract(r"((?<= )[^/]+)", expand=False)
df_limpo['genre'] = genre

#developer
dev = df['summary_detail_developer'].str.extract(r"((?<=Developer:\n\n).*)", expand=False)
df_limpo['developer'] = dev

#publisher
pub = df['summary_detail_publisher'].str.extract("((?<=Publisher:\n).*)", expand=False)
df_limpo['publisher'] = pub


#rating
rat = df['summary_detail_product_rating'].str.extract("((?<=Rating:\n).*)", expand=False)
df_limpo['rating'] = rat

df_limpo['y'] = df['y']

#dropingna
#Due to the low presence of null values ​​we can simply drop them without harming the process as a whole.
df_limpo_nonull = df_limpo.dropna()




## Below we can see the dataframe with the clean data.

In [ ]:
df_limpo_nonull.sample(5)

,release_data,tittle,userscore,metascore,genre,developer,publisher,rating,y
117,2013-11-15,Deadfall Adventures,7.7,43,"Action, Shooter, Shooter, First-Person, Fantas...",The Farm 51,Nordic Games Publishing,M,0.0
225,2009-12-23,0 Day Attack on Earth,4.5,41,"Action, Shooter, Shooter, Third-Person, Sci-Fi...",Gulti,Square Enix,E10+,0.0
262,2014-11-18,Escape Dead Island,3.1,40,"Action Adventure, Adventure, 3D, Survival, Thi...","Fatshark AB, Fatshark",Deep Silver,M,0.0
665,2003-11-06,Prince of Persia: The Sands of Time,8.8,92,"Action Adventure, Fantasy",Ubisoft,Ubisoft,T,1.0
814,2003-08-12,Madden NFL 2004,8.3,94,"Sports, Traditional, Football, Sim",EA Tiburon,EA Sports,E,1.0
